In [2]:
# crypto_agent.py
!pip install ccxt
import ccxt
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
import joblib

# Fetch crypto data
def fetch_crypto_data(symbol='BTC/USDT', timeframe='1h', limit=100):
    exchange = ccxt.kraken()
    try:
        ohlcv = exchange.fetch_ohlcv(symbol, timeframe, limit=limit)
        df = pd.DataFrame(ohlcv, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])
        df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
        return df
    except Exception as e:
        print(f"Error fetching data: {e}")
        return None

# Add features
def add_features(df):
    df['sma_10'] = df['close'].rolling(window=10).mean()
    df['sma_50'] = df['close'].rolling(window=50).mean()
    delta = df['close'].diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=14).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=14).mean()
    rs = gain / loss
    df['rsi'] = 100 - (100 / (1 + rs))
    return df.dropna()

# Prepare data
def prepare_data(df):
    df['target'] = (df['close'].shift(-1) > df['close']).astype(int)
    df = df.dropna()
    X = df[['sma_10', 'sma_50', 'rsi', 'close', 'volume']]
    y = df['target']
    return X, y, df

# Train and save model
def train_model():
    data = fetch_crypto_data()
    if data is None:
        return None
    data = add_features(data)
    X, y, processed_data = prepare_data(data)
    model = RandomForestClassifier(n_estimators=100, random_state=42)
    model.fit(X, y)
    joblib.dump(model, 'crypto_model.pkl')
    return model, processed_data

# Predict latest
def predict_latest():
    model = joblib.load('crypto_model.pkl')
    latest_data = fetch_crypto_data()
    if latest_data is None:
        return None, None, None
    latest_data = add_features(latest_data)
    latest_features = latest_data[['sma_10', 'sma_50', 'rsi', 'close', 'volume']].iloc[-1:]
    prediction = model.predict(latest_features)[0]
    action = "Buy" if prediction == 1 else "Sell"
    price = latest_data['close'].iloc[-1]
    return action, price, latest_data

if __name__ == "__main__":
    model, processed_data = train_model()
    if model:
        action, price, data = predict_latest()
        if action:
            print(f"Latest Price: ${price:.2f}")
            print(f"Recommended Action: {action}")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.0/131.0 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.7/5.7 MB 51.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 49.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.3/289.3 kB 18.0 MB/s eta 0:00:00
  Attempting uninstall: aiohttp
    Found existing installation: aiohttp 3.11.15
    Uninstalling aiohttp-3.11.15:
      Successfully uninstalled aiohttp-3.11.15
Latest Price: $85300.40
Recommended Action: Sell


Exported Colab notebook to a local project folder, preparing code for Flask integration.